<a href="https://colab.research.google.com/github/AbsolutUnit/Textract-Pipeline/blob/main/config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary Setup

In [1]:
!pip install boto3
!pip install awscli
!pip install PyMuPDF
!pip install trp
!pip install textract-trp
!pip install pdf2image
!pip install python-poppler

  Using cached python-poppler-0.2.2.tar.gz (595 kB)
  ERROR: Failed building wheel for python-poppler
  Running setup.py clean for python-poppler
Failed to build python-poppler
    Running setup.py install for python-poppler ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-z7vfkfec/python-poppler_9bcb29e969aa47eea0df36c545362b79/setup.py'"'"'; __file__='"'"'/tmp/pip-install-z7vfkfec/python-poppler_9bcb29e969aa47eea0df36c545362b79/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-20mb_ltp/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/python-poppler Check the logs

In [2]:
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [3]:
import os
import cv2
import csv
import cv2
import fitz
import json
import numpy as np
from trp import Document
import matplotlib.pyplot as plt
from matplotlib import gridspec
from pdf2image import convert_from_path

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [5]:
os.chdir('gdrive/My Drive')

In [6]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [7]:
# def visualize_hp(file_name, img, row_mw_cutpoints):
#     row_highlight = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     row_highlight[row_m == 0, :, :] = [255,191,191]
#     row_highlight[row_split :] = [255,0,0]
#     plot_horizontal_projection(file_name, row_highlight, row_m)

# def visualize_vp(file_name, img, col_mn_cutpoints):
#     col_highlight = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     col_highlight[:, col_m == 0, :] = [255,191,191]
#     col_highlight[:, col_split[255,0,0]
#     plot_vertical_projection(file_name, col_highlight, col_m)

In [8]:
# def plot_horizontal_projection(file_name, img, projection):
#     fig = plt.figure(1, figsize=(12,16))
#     gs = gridspec.GridSpec(1, 2, width_ratios=[3,1])

#     ax = plt.subplot(gs[0])
#     im = ax.imshow(img, interpolation='nearest', aspect='auto')
#     ax.grid(which='major', alpha=0.5)

#     ax = plt.subplot(gs[1])
#     ax.plot(projection, np.arange(img.shape[0]), 'm')
#     ax.grid(which='major', alpha=0.5)
#     plt.xlim([0.0, 255.0])
#     plt.ylim([-0.5, img.shape[0] - 0.5])
#     ax.invert_yaxis()

#     fig.suptitle("FOO", fontsize=16)
#     gs.tight_layout(fig, rect=[0, 0.03, 1, 0.97])

#     fig.set_dpi(200)

#     fig.savefig(file_name, bbox_inches='tight', dpi=fig.dpi)
#     plt.clf()

In [9]:
# def plot_vertical_projection(file_name, img, projection):
#     fig = plt.figure(2, figsize=(12, 4))
#     gs = gridspec.GridSpec(2, 1, height_ratios=[1,5])

#     ax = plt.subplot(gs[0])
#     im = ax.imshow(img, interpolation='nearest', aspect='auto')
#     ax.grid(which='major', alpha=0.5)

#     ax = plt.subplot(gs[1])
#     ax.plot(np.arange(img.shape[1]), projection, 'm')
#     ax.grid(which='major', alpha=0.5)
#     plt.xlim([-0.5, img.shape[1] - 0.5])
#     plt.ylim([0.0, 255.0])

#     fig.suptitle("FOO", fontsize=16)
#     gs.tight_layout(fig, rect=[0, 0.03, 1, 0.97])

#     fig.set_dpi(200)

#     fig.savefig(file_name, bbox_inches='tight', dpi=fig.dpi)
#     plt.clf()

This opens the file descriptor for the JSON file, which is from the Textract Lambda Function. Then, the directory gets changed to the CSV folder, for CSV output.

In [10]:
f = open('sent_file (3).json')

In [11]:
os.chdir('/content/gdrive/My Drive/Textract Demo/CSV Output')

The following is the CSV conversion function, which looks through all tables and cells to compile a list of cells and output them into a CSV file.

In [12]:
t_doc = Document(json.load(f))
with open('exported_json.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',')
    for page in t_doc.pages:
        for table in page.tables:
            for r, row in enumerate(table.rows):
                row_ex = []
                for c, cell in enumerate(row.cells):
                    row_ex.append(cell.text.strip())
                writer.writerow(row_ex)
            writer.writerow([])
f.close()

In [13]:
os.chdir('/content/gdrive/My Drive')

Change to main directory to obtain another file descriptor for the JSON file.

In [14]:
f2 = open('sent_file (3).json')

In [15]:
os.chdir('/content/gdrive/My Drive/Textract Demo/JSON Output')

Walks through the JSON file's parameters and compiles tables to append to a list. This list is then appended to the pages list and dumped into a JSON file as output.

In [16]:
t_doc = Document(json.load(f2))
pages = []
for page in t_doc.pages:
  tables = []
  for table in page.tables:
    table_ex = []
    for r, row in enumerate(table.rows):
      row_ex = []
      for c, cell in enumerate(row.cells):
          row_ex.append(cell.text.strip())
      if row_ex:
        table_ex.append(row_ex)
    if table:
      tables.append(table_ex)
  if tables:
    pages.append(tables)
print(pages)
ret_js = json.dumps(pages)
with open("export_json.json", "w") as outfile:
  outfile.write(ret_js)

[[], [], [[['', '', 'New Estimates', '', '%', 'change (YoY)', '', 'Old', 'Estimates', '', '% change', '(Newvs.', 'Old)'], ['', '2017E', '2018E', '2019E', '17E/16A', '18E/17E', '19E/18E', '2017E', '2018E', '2019E', '2017E', '2018E', '2019E'], ['Net Revenues', '33,655', '32,781', '34,013', '16%', '-3%', '4%', '31,711', '32,484', '33,664', '6%', '1%', '1%'], ['Cash COGS', '17,506', '17,234', '17,596', '10%', '-2%', '2%', '17,529', '17,244', '17,817', '0%', '0%', '-1%'], ['EBITDA', '14,819', '14,058', '14,987', '26%', '-5%', '7%', '12,886', '13,740', '14,403', '15%', '2%', '4%'], ['EBITDA Margin', '44%', '43%', '44%', '3.4 pp', ', -1.1 pp', '1.2 pp', '41%', '42%', '43%', ', 3.4 pp', '0.6 pp', ', 1.3 pp'], ['Net Income', '6,611', '5,288', '7,031', '180%', '-20%', '33%', '4,721', '5,094', '6,530', '40%', '4%', '8%'], ['EPS', '1.31', '1.03', '1.36', '86%', '-21%', '33%', '0.94', '0.99', '1.27', '39%', '4%', '8%'], ['Capex', '4,427', '4,473', '4,001', '-19%', '1%', '-11%', '4,451', '4,417', '3

In [17]:
os.chdir('/content/gdrive/My Drive')

Here, we are converting the pages of the pdf to images.

In [18]:
image = convert_from_path("sent_file.pdf")
i = 0

In [19]:
os.chdir('/content/gdrive/My Drive/Textract Demo/Graphs Output')

This code iterates through each page and saves it in the Graphs folder as a PNG file.

In [20]:
i = 0
for img in image:
  print("Hi")
  img.save('Img_Out' + str(i) + '\out.png', 'PNG')
  i += 1
res = []


Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi


This bulk of code is the main OpenCV usage.
In this, borderlines are drawn horizontally and vertically, creating cutpoints along the document. This helps to highlight the sectioned parts and break up the image PDF into segments to find the larger bounding boxes.

In [21]:
for i in range(0, 12):
  
  img = cv2.imread('Img_Out' + str(i) + '\out.png', cv2.IMREAD_COLOR)
  img_gray_inverted = 255 - cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  row_m = cv2.reduce(img_gray_inverted, 1, cv2.REDUCE_AVG, dtype=cv2.CV_32F).flatten()
  row_g = zero_runs(row_m)
  row_split = (row_g[:,0] + row_g[:,1] - 1) / 2
  row_split = row_split.astype(int)

  #visualize_hp("Img_Out8\out_hp.png", img, row_m, row_split)

  bbox = []
  for n,(start,end) in enumerate(zip(row_split, row_split[1:])):
      line = img[start:end]
      line_gray_inverted = img_gray_inverted[start:end]

      col_m = cv2.reduce(line_gray_inverted, 0, cv2.REDUCE_AVG, dtype=cv2.CV_32F).flatten()
      col_g = zero_runs(col_m)
      column_gap_sizes = col_g[:,1] - col_g[:,0]
      col_split = (col_g[:,0] + col_g[:,1] - 1) / 2
      col_split = col_split.astype(int)

      filtered_cutpoints = col_split[column_gap_sizes > 5]

      for xstart,xend in zip(filtered_cutpoints, filtered_cutpoints[1:]):
          bbox.append(((xstart, start), (xend, end)))

      #visualize_vp("article_vp_%02d.png" % n, line, col_m, filtered_cutpoints)

  result = img.copy()

  for bounding_box in bbox:
      cv2.rectangle(result, bounding_box[0], bounding_box[1], (255,0,0), 2)

  if result.any():
    cv2.imwrite("article_boxes_page_" + str(i) + ".png", result)
  else:
    continue
    
for i in range(0, 12):
  os.remove('Img_Out' + str(i) + '\out.png')

# TODO
Find a method of accurately pulling most, if not all graphs and tables. This means finding the threshold for filtering them out from lines of text, but this becomes problematic when tables are arranged in a non-normal manner. Borderless tables, lineless tables, and Excel-cell like tables with large cell widths are problematic since they cause the bounding box to be split into multiple smaller ones, which may be filtered by the size threshold.